In [1]:
import datetime as dt
import os
import sys

import numpy as np
import pandas as pd
from scipy import interp
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, confusion_matrix, roc_curve, average_precision_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold
import sqlalchemy as sa
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../')
from utilities import sql_utils as su
from utilities import model_eval_utils as meu

DWH = os.getenv('MIMIC_DWH')
engine = create_engine(DWH)

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/Users/VincentLa/anaconda3/envs/cse6250-project/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
QUERY = """
select
  subject_id,
  hadm_id,
  chartdate,
  text
from mimiciii.noteevents
limit 200000
"""
with engine.connect() as conn:
    df = pd.read_sql(QUERY, conn)

In [3]:
df.shape

(200000, 4)

In [4]:
df.head()

,subject_id,hadm_id,chartdate,text
0,14139,114588.000,2198-06-06,[**2198-6-6**] 4:00 PM\n CHEST (PORTABLE AP); ...
1,1563,nan,2172-03-18,[**2172-3-18**] 4:00 PM\n CHEST (PA & LAT) ...
2,8182,nan,2194-04-16,[**2194-4-16**] 12:04 PM\n ART DUP EXT LO UNI;...
3,8297,113537.000,2115-05-01,[**2115-5-1**] 12:54 PM\n RENAL TRANSPLANT U.S...
4,20473,nan,2126-05-29,[**2126-5-29**] 12:56 PM\n CT HEAD W/ & W/O CO...


In [5]:
data_text = df[['text']]
data_text['index'] = data_text.index

/Users/VincentLa/anaconda3/envs/cse6250-project/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
data_text.head()

,text,index
0,[**2198-6-6**] 4:00 PM\n CHEST (PORTABLE AP); ...,0
1,[**2172-3-18**] 4:00 PM\n CHEST (PA & LAT) ...,1
2,[**2194-4-16**] 12:04 PM\n ART DUP EXT LO UNI;...,2
3,[**2115-5-1**] 12:54 PM\n RENAL TRANSPLANT U.S...,3
4,[**2126-5-29**] 12:56 PM\n CT HEAD W/ & W/O CO...,4


In [7]:
documents = data_text

In [8]:
documents.head()

,text,index
0,[**2198-6-6**] 4:00 PM\n CHEST (PORTABLE AP); ...,0
1,[**2172-3-18**] 4:00 PM\n CHEST (PA & LAT) ...,1
2,[**2194-4-16**] 12:04 PM\n ART DUP EXT LO UNI;...,2
3,[**2115-5-1**] 12:54 PM\n RENAL TRANSPLANT U.S...,3
4,[**2126-5-29**] 12:56 PM\n CT HEAD W/ & W/O CO...,4


In [9]:
print(len(documents))
print(documents[:5])

200000
                                                text  index
0  [**2198-6-6**] 4:00 PM\n CHEST (PORTABLE AP); ...      0
1  [**2172-3-18**] 4:00 PM\n CHEST (PA & LAT)    ...      1
2  [**2194-4-16**] 12:04 PM\n ART DUP EXT LO UNI;...      2
3  [**2115-5-1**] 12:54 PM\n RENAL TRANSPLANT U.S...      3
4  [**2126-5-29**] 12:56 PM\n CT HEAD W/ & W/O CO...      4


## Data Preprocessing

1. Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
2. Words that have fewer than 3 characters are removed.
3. All stopwords are removed.
4. Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
5. Words are stemmed — words are reduced to their root form.

### Loading gensim and nltk libraries

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
# import nltk.stem as stemmer
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

stemmer = SnowballStemmer('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/VincentLa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def lemmatize_stemming(text):
    """
    Lemmatize: lemmatized — words in third person are changed to first person
    
    Verbs in past and future tenses are changed into present.
    """
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [12]:
def preprocess(text):
    """
    Preprocess Text:
    
    Remove words in "STOPWORDS" and remove words 3 letters or less
    """
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [13]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')

words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['[**2163-9-22**]', '4:41', 'PM\n', 'CT', 'C-SPINE', 'W/CONTRAST;', 'CT', '100CC', 'NON', 'IONIC', 'CONTRAST', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Clip', '#', '[**Clip', 'Number', '(Radiology)', '56611**]\n', 'CT', 'RECONSTRUCTION\n', 'Reason:', '?fluid', 'collection\n', '', 'Contrast:', 'OPTIRAY', 'Amt:', '100\n', '______________________________________________________________________________\n', '[**Hospital', '2**]', 'MEDICAL', 'CONDITION:\n', '', '52', 'year', 'old', 'man', 'with', 'cervical', 'fx/meningitis', 'now', 'w/AMS\n', 'REASON', 'FOR', 'THIS', 'EXAMINATION:\n', '', '?fluid', 'collection\n', 'No', 'contraindications', 'for', 'IV', 'contrast\n', '______________________________________________________________________________\n', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'FINAL', 'REPORT\n', 'HISTORY:', '', 'Status', 'post', 'cervical', 'spine', 'fractures

In [15]:
processed_docs = documents['text'].map(preprocess)
processed_docs[:10]

0    [chest, portabl, differ, physician, initi, cli...
1    [chest, clip, clip, number, radiolog, reason, ...
2    [clip, clip, number, radiolog, reason, arterio...
3    [renal, transplant, right, clip, clip, number,...
4    [head, contrast, ionic, contrast, clip, clip, ...
5    [chest, portabl, clip, clip, number, radiolog,...
6    [chest, portabl, clip, clip, number, radiolog,...
7    [chest, recon, nonion, contrast, clip, clip, n...
8    [chest, portabl, clip, clip, number, radiolog,...
9    [head, recon, neck, recon, clip, clip, number,...
Name: text, dtype: object

## Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [16]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 accid
1 admit
2 aspect
3 bilater
4 brachiocephal
5 cardiomegali
6 cathet
7 central
8 chang
9 chest
10 clip


### Gensim filter_extremes
Filter out tokens that appear in

1. less than 15 documents (absolute number) or
2. more than 0.5 documents (fraction of total corpus size, not absolute number).
3. after the above two steps, keep only the first 100000 most frequent tokens.

In [17]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## Gensim doc2bow (Bag of Words)
For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(2, 2),
 (26, 1),
 (28, 1),
 (30, 1),
 (36, 2),
 (37, 1),
 (46, 2),
 (49, 1),
 (53, 1),
 (60, 1),
 (61, 2),
 (66, 1),
 (78, 4),
 (81, 4),
 (86, 1),
 (101, 1),
 (103, 1),
 (113, 9),
 (122, 8),
 (129, 2),
 (138, 1),
 (142, 1),
 (154, 2),
 (157, 1),
 (182, 1),
 (193, 4),
 (194, 2),
 (197, 1),
 (211, 1),
 (212, 1),
 (215, 2),
 (224, 1),
 (226, 1),
 (229, 2),
 (233, 1),
 (238, 1),
 (241, 1),
 (293, 1),
 (306, 2),
 (322, 1),
 (326, 1),
 (335, 1),
 (336, 1),
 (338, 1),
 (367, 2),
 (395, 1),
 (397, 1),
 (401, 2),
 (438, 2),
 (481, 2),
 (512, 1),
 (520, 2),
 (592, 1),
 (613, 5),
 (677, 1),
 (687, 5),
 (689, 2),
 (730, 1),
 (744, 1),
 (750, 1),
 (759, 2),
 (893, 1),
 (900, 4),
 (909, 2),
 (985, 1),
 (1252, 1),
 (1460, 1),
 (1704, 1),
 (2008, 1),
 (2641, 2),
 (2992, 2),
 (3331, 2)]

In [19]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 2 ("bilater") appears 2 time.
Word 26 ("obtain") appears 1 time.
Word 28 ("persist") appears 1 time.
Word 30 ("place") appears 1 time.
Word 36 ("post") appears 2 time.
Word 37 ("present") appears 1 time.
Word 46 ("status") appears 2 time.
Word 49 ("techniqu") appears 1 time.
Word 53 ("tube") appears 1 time.
Word 60 ("evalu") appears 1 time.
Word 61 ("evid") appears 2 time.
Word 66 ("histori") appears 1 time.
Word 78 ("soft") appears 4 time.
Word 81 ("tissu") appears 4 time.
Word 86 ("appear") appears 1 time.
Word 101 ("perform") appears 1 time.
Word 103 ("region") appears 1 time.
Word 113 ("collect") appears 9 time.
Word 122 ("fluid") appears 8 time.
Word 129 ("imag") appears 2 time.
Word 138 ("note") appears 1 time.
Word 142 ("patient") appears 1 time.
Word 154 ("slight") appears 2 time.
Word 157 ("suggest") appears 1 time.
Word 182 ("axial") appears 1 time.
Word 193 ("contrast") appears 4 time.
Word 194 ("decreas") appears 2 time.
Word 197 ("enhanc") appears 1 time.
Word 211 ("i

## TF IDF 
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [20]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.11030306282395226),
 (1, 0.08673954089788365),
 (2, 0.03700529163661585),
 (3, 0.11046351873995343),
 (4, 0.07731225353127807),
 (5, 0.045467672917461115),
 (6, 0.06242559423081927),
 (7, 0.057182843407501814),
 (8, 0.09500468637945848),
 (9, 0.04024977223114698),
 (10, 0.05607448307304172),
 (11, 0.10770107514461968),
 (12, 0.283436484636098),
 (13, 0.13704235129763967),
 (14, 0.07337142875145794),
 (15, 0.023844263991027976),
 (16, 0.11749752866354181),
 (17, 0.08271062345748974),
 (18, 0.07559844341156134),
 (19, 0.10444690814593954),
 (20, 0.019691007529958768),
 (21, 0.08173775294278883),
 (22, 0.053918566816367836),
 (23, 0.04234181468510944),
 (24, 0.10331468548377692),
 (25, 0.1674829093348646),
 (26, 0.042258577768104946),
 (27, 0.04831118488674307),
 (28, 0.06317412621916588),
 (29, 0.061908829542932614),
 (30, 0.0556960325621494),
 (31, 0.1767185083071269),
 (32, 0.026659550435654534),
 (33, 0.03827841279468709),
 (34, 0.025232773775803883),
 (35, 0.092943573431235),


## Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [21]:
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=10,
                                       id2word=dictionary,
                                       passes=2,
                                       workers=2)

In [22]:
type(lda_model)

gensim.models.ldamulticore.LdaMulticore

For each topic, we will explore the words occuring in that topic and its relative weight.

In [23]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.041*"vein" + 0.028*"liver" + 0.018*"hepat" + 0.017*"flow" + 0.015*"lower" + 0.014*"portal" + 0.014*"normal" + 0.013*"evalu" + 0.013*"extrem" + 0.012*"ascit"
Topic: 1 
Words: 0.047*"contrast" + 0.016*"pelvi" + 0.015*"abdomen" + 0.013*"small" + 0.011*"imag" + 0.011*"fluid" + 0.010*"lesion" + 0.009*"note" + 0.008*"appear" + 0.008*"see"
Topic: 2 
Words: 0.031*"procedur" + 0.022*"cathet" + 0.021*"identifi" + 0.020*"numer" + 0.020*"patient" + 0.019*"place" + 0.012*"remov" + 0.012*"picc" + 0.011*"perform" + 0.011*"wire"
Topic: 3 
Words: 0.032*"contrast" + 0.017*"mass" + 0.017*"head" + 0.015*"imag" + 0.014*"brain" + 0.011*"lesion" + 0.010*"enhanc" + 0.010*"evid" + 0.009*"infarct" + 0.009*"arteri"
Topic: 4 
Words: 0.031*"spine" + 0.025*"arteri" + 0.020*"carotid" + 0.020*"imag" + 0.015*"cervic" + 0.015*"vertebr" + 0.014*"stenosi" + 0.012*"namepattern" + 0.011*"level" + 0.010*"mild"
Topic: 5 
Words: 0.034*"bowel" + 0.031*"abdomen" + 0.027*"obstruct" + 0.020*"abdomin" + 0.018*"f

## Running LDA using TF-IDF


In [24]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.020*"dobhoff" + 0.016*"dobbhoff" + 0.013*"ercp" + 0.013*"tube" + 0.012*"paracentesi" + 0.011*"sicu" + 0.011*"placement" + 0.010*"biliari" + 0.010*"feed" + 0.009*"duct"
Topic: 1 Word: 0.023*"fractur" + 0.014*"knee" + 0.012*"joint" + 0.010*"femur" + 0.010*"shoulder" + 0.010*"foot" + 0.009*"ankl" + 0.008*"babygram" + 0.008*"view" + 0.008*"disloc"
Topic: 2 Word: 0.017*"liver" + 0.015*"hepat" + 0.014*"portal" + 0.011*"flow" + 0.011*"vein" + 0.010*"ascit" + 0.009*"transplant" + 0.009*"doppler" + 0.008*"swallow" + 0.008*"cirrhosi"
Topic: 3 Word: 0.035*"picc" + 0.020*"line" + 0.015*"placement" + 0.011*"procedur" + 0.011*"numer" + 0.010*"place" + 0.010*"cathet" + 0.007*"sheath" + 0.007*"access" + 0.007*"port"
Topic: 4 Word: 0.012*"tube" + 0.010*"interv" + 0.009*"portabl" + 0.009*"effus" + 0.009*"pneumonia" + 0.009*"placement" + 0.009*"chang" + 0.008*"pleural" + 0.008*"pneumothorax" + 0.008*"lung"
Topic: 5 Word: 0.023*"bowel" + 0.020*"obstruct" + 0.018*"abdomen" + 0.015*"free" +

## Performance evaluation by classifying sample document using LDA Bag of Words model

In [26]:
processed_docs[4310]

['spine',
 'contrast',
 'ionic',
 'contrast',
 'clip',
 'clip',
 'number',
 'radiolog',
 'reconstruct',
 'reason',
 'fluid',
 'collect',
 'contrast',
 'optiray',
 'hospit',
 'medic',
 'condit',
 'year',
 'cervic',
 'mening',
 'reason',
 'examin',
 'fluid',
 'collect',
 'contrast',
 'final',
 'report',
 'histori',
 'status',
 'post',
 'cervic',
 'spine',
 'fractur',
 'mening',
 'evalu',
 'fluid',
 'collect',
 'comparison',
 'cervic',
 'spine',
 'cervic',
 'spine',
 'techniqu',
 'axial',
 'multidetector',
 'imag',
 'cervic',
 'spine',
 'obtain',
 'intraven',
 'optiray',
 'sagitt',
 'coron',
 'reformat',
 'imag',
 'perform',
 'year',
 'digit',
 'evid',
 'laminectomi',
 'anterior',
 'fusion',
 'anatom',
 'align',
 'present',
 'fractur',
 'spinous',
 'process',
 'visual',
 'persist',
 'swell',
 'prevertebr',
 'soft',
 'tissu',
 'level',
 'fusion',
 'slight',
 'decreas',
 'definit',
 'fluid',
 'collect',
 'region',
 'fluid',
 'collect',
 'posterior',
 'paraspin',
 'soft',
 'tissu',
 'level',

In [27]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.36354631185531616	 
Topic: 0.047*"contrast" + 0.016*"pelvi" + 0.015*"abdomen" + 0.013*"small" + 0.011*"imag" + 0.011*"fluid" + 0.010*"lesion" + 0.009*"note" + 0.008*"appear" + 0.008*"see"

Score: 0.30859193205833435	 
Topic: 0.031*"spine" + 0.025*"arteri" + 0.020*"carotid" + 0.020*"imag" + 0.015*"cervic" + 0.015*"vertebr" + 0.014*"stenosi" + 0.012*"namepattern" + 0.011*"level" + 0.010*"mild"

Score: 0.23450230062007904	 
Topic: 0.030*"fractur" + 0.024*"hemorrhag" + 0.023*"contrast" + 0.019*"head" + 0.015*"chang" + 0.011*"acut" + 0.010*"sinus" + 0.010*"hematoma" + 0.009*"evid" + 0.008*"intracrani"

Score: 0.05471209064126015	 
Topic: 0.032*"contrast" + 0.017*"mass" + 0.017*"head" + 0.015*"imag" + 0.014*"brain" + 0.011*"lesion" + 0.010*"enhanc" + 0.010*"evid" + 0.009*"infarct" + 0.009*"arteri"

Score: 0.03477079048752785	 
Topic: 0.090*"tube" + 0.031*"placement" + 0.022*"portabl" + 0.021*"pneumothorax" + 0.020*"posit" + 0.013*"endotrach" + 0.013*"post" + 0.012*"remov" + 0.012*"

## Performance evaluation by classifying sample document using LDA TF-IDF model.

In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5306692719459534	 
Topic: 0.025*"spine" + 0.022*"fractur" + 0.014*"cervic" + 0.010*"trauma" + 0.010*"spinal" + 0.009*"lumbar" + 0.009*"fusion" + 0.009*"disc" + 0.008*"injuri" + 0.008*"cord"

Score: 0.27951470017433167	 
Topic: 0.017*"contrast" + 0.010*"pelvi" + 0.008*"abdomen" + 0.006*"kidney" + 0.006*"lesion" + 0.006*"renal" + 0.006*"fluid" + 0.006*"measur" + 0.005*"liver" + 0.005*"imag"

Score: 0.1843886524438858	 
Topic: 0.019*"hemorrhag" + 0.017*"head" + 0.017*"contrast" + 0.011*"intracrani" + 0.009*"sinus" + 0.009*"frontal" + 0.009*"brain" + 0.009*"carotid" + 0.009*"mass" + 0.008*"infarct"


## Testing model on unseen document


In [34]:
unseen_document = """
Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]


Service:
ADDENDUM:

RADIOLOGIC STUDIES:  Radiologic studies also included a chest
CT, which confirmed cavitary lesions in the left lung apex
consistent with infectious process/tuberculosis.  This also
moderate-sized left pleural effusion.

HEAD CT:  Head CT showed no intracranial hemorrhage or mass
effect, but old infarction consistent with past medical
history.

ABDOMINAL CT:  Abdominal CT showed lesions of
T10 and sacrum most likely secondary to osteoporosis. These can
be followed by repeat imaging as an outpatient.



                            [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]

Dictated By:[**Hospital 1807**]
MEDQUIST36

D:  [**2151-8-5**]  12:11
T:  [**2151-8-5**]  12:21
JOB#:  [**Job Number 1808**]

"""
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4775536060333252	 Topic: 0.032*"contrast" + 0.017*"mass" + 0.017*"head" + 0.015*"imag" + 0.014*"brain"
Score: 0.22977572679519653	 Topic: 0.036*"effus" + 0.029*"pleural" + 0.023*"lung" + 0.021*"chang" + 0.019*"portabl"
Score: 0.16698352992534637	 Topic: 0.031*"spine" + 0.025*"arteri" + 0.020*"carotid" + 0.020*"imag" + 0.015*"cervic"
Score: 0.11368417739868164	 Topic: 0.034*"bowel" + 0.031*"abdomen" + 0.027*"obstruct" + 0.020*"abdomin" + 0.018*"free"
